# Actividad Guiada 2 (Extr): DengAI. Predicting Disease Spread - Predicción

__Objetivos:__ Predicción sobre los casos de dengue en una semana a partir de los datos metereológicos de la misma. La competición mide el error cometido en la predicción.

__Técnica a utilizar:__ Técnicas de predicción de las vistas en la asignatura.

__Tecnología:__ Libre, debe justificarse en la memoria a entregar.

__Entregable:__ Los alumnos realizarán una memoria (límite 10 páginas) describiendo lo siguiente: el problema afrontado, las técnicas utilizadas y su parametrización y los resultados obtenidos incluyendo el usuario de DrivenData utilizado para obtenerlo. Esta memoria contendrá explicaciones sobre los códigos desarrollados.

In [ ]:
import numpy as np
import pandas as pd


In [1]:
train = pd.read_csv('./Data/dengue_features_train.csv', encoding='utf-8')
test = pd.read_csv('./Data/dengue_features_test.csv', encoding='utf-8')
labels = pd.read_csv('./Data/dengue_labels_train.csv', encoding='utf-8')



NameError: name 'jj' is not defined